In [1]:
from List_to_Pytorch_Dataset import dataset
import torch
from torch import nn
from torch.utils.data import DataLoader
import CNN1D
import Smaller_input_CNN
from statistics import mean

In [2]:
print(f"Dataset size: {len(dataset)}")
print(f"Data sample size: {len(dataset[0][0][0])}")

Dataset size: 2797
Data sample size: 1000


In [3]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda:0 device


In [4]:
labels = { 0  : "STD",
           1  : "WAL",
           2  : "JOG",
           3  : "JUM",
           4  : "STU",
           5  : "STN",
           6  : "SCH",
           7  : "SIT",
           8  : "CHU",
           9  : "CSI",
           10 : "CSO",
           11 : "FOL",
           12 : "FKL",
           13 : "BSC",
           14 : "SDL"}

classification_map = {}

for key, value in labels.items():
    classification_map[value] = {"TP" : 0,
                                 "FP" : 0,
                                 "TN" : 0,
                                 "FN" : 0}

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch%10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [6]:
def valid_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    valid_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            
            pred = model(X)

            for key, value in labels.items():
                for i in range(len(y)):
                    if y[i] == key:
                        if pred[i].argmax() == key:
                            classification_map[value]["TP"] += 1
                        else:
                            classification_map[value]["FN"] += 1
                    elif pred[i].argmax() == key:
                        classification_map[value]["FP"] += 1
                    else:
                        classification_map[value]["TN"] += 1
                

            
            
            #model loss and accuracy
            valid_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    #model stats
    valid_loss /= num_batches
    correct /= size
    print(f"Validation Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {valid_loss:>8f} \n\n")
    return 100*correct

In [7]:
def main():
    model = CNN1D.CNN().to(device)
    model.train()

    learning_rate = 0.001
    epochs = 50

    # Initialize the loss function and optimizer
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    #creating datasets
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])

    #Dataloaders for the datasets
    train_dataloader = DataLoader(train_set, batch_size = 64, shuffle = True)
    test_dataloader = DataLoader(test_set, batch_size = 64, shuffle = True)
    
    model.train()
    for _ in range(epochs):
        train_loop(train_dataloader, model, loss_fn, optimizer)
    
    model.eval()

    model_accuracy = valid_loop(test_dataloader, model, loss_fn)

    return model_accuracy


In [8]:
accuracies = []
for _ in range(10):
    accuracy = main()
    accuracies.append(accuracy)

print(f"Average accuracy: {mean(accuracies)}")


loss: 2.873249  [    0/ 2237]
loss: 2.678178  [  640/ 2237]
loss: 2.674041  [ 1280/ 2237]
loss: 2.637007  [ 1920/ 2237]
loss: 2.424213  [    0/ 2237]
loss: 2.138942  [  640/ 2237]
loss: 2.246877  [ 1280/ 2237]
loss: 1.767235  [ 1920/ 2237]
loss: 1.541805  [    0/ 2237]
loss: 1.907111  [  640/ 2237]
loss: 1.616918  [ 1280/ 2237]
loss: 1.508684  [ 1920/ 2237]
loss: 1.339910  [    0/ 2237]
loss: 1.537681  [  640/ 2237]
loss: 1.369667  [ 1280/ 2237]
loss: 1.348108  [ 1920/ 2237]
loss: 1.266605  [    0/ 2237]
loss: 1.123954  [  640/ 2237]
loss: 1.289356  [ 1280/ 2237]
loss: 1.327347  [ 1920/ 2237]
loss: 1.314199  [    0/ 2237]
loss: 1.296201  [  640/ 2237]
loss: 0.926515  [ 1280/ 2237]
loss: 1.083593  [ 1920/ 2237]
loss: 1.239428  [    0/ 2237]
loss: 1.040623  [  640/ 2237]
loss: 1.026773  [ 1280/ 2237]
loss: 1.023646  [ 1920/ 2237]
loss: 0.861920  [    0/ 2237]
loss: 0.552625  [  640/ 2237]
loss: 1.132398  [ 1280/ 2237]
loss: 0.624012  [ 1920/ 2237]
loss: 0.667154  [    0/ 2237]
loss: 0.90

In [9]:
stats_map = {}
for key, value in labels.items():
    stats_map[value] = {
        "Sensitivity" : float(classification_map[value]["TP"]) / float(classification_map[value]["TP"] + classification_map[value]["FN"]),
        "Specificity" : float(classification_map[value]["TN"]) / float(classification_map[value]["TN"] + classification_map[value]["FP"]),
        "Recall" : float(classification_map[value]["TP"]) / float(classification_map[value]["TP"] + classification_map[value]["FN"]),
        "Precision" : float(classification_map[value]["TP"]) / float(classification_map[value]["TP"] + classification_map[value]["FP"]),
        "Accuracy" : float(classification_map[value]["TP"] + classification_map[value]["TN"]) / float(classification_map[value]["TP"] + classification_map[value]["TN"] + classification_map[value]["FP"] + classification_map[value]["FN"])
    }
    stats_map[value]["F-score"] = 2.0 / float(1.0 / float(stats_map[value]["Precision"]) + 1.0 / float(stats_map[value]["Recall"]))
print(stats_map)

{'STD': {'Sensitivity': 0.9554317548746518, 'Specificity': 0.9891242129364625, 'Recall': 0.9554317548746518, 'Precision': 0.8575, 'Accuracy': 0.9869642857142857, 'F-score': 0.903820816864295}, 'WAL': {'Sensitivity': 0.7123287671232876, 'Specificity': 0.9816618911174785, 'Recall': 0.7123287671232876, 'Precision': 0.7303370786516854, 'Accuracy': 0.9641071428571428, 'F-score': 0.7212205270457698}, 'JOG': {'Sensitivity': 0.8411458333333334, 'Specificity': 0.9921395705521472, 'Recall': 0.8411458333333334, 'Precision': 0.8873626373626373, 'Accuracy': 0.9817857142857143, 'F-score': 0.8636363636363636}, 'JUM': {'Sensitivity': 0.9202279202279202, 'Specificity': 0.9923795008573062, 'Recall': 0.9202279202279202, 'Precision': 0.8898071625344353, 'Accuracy': 0.9878571428571429, 'F-score': 0.9047619047619047}, 'STU': {'Sensitivity': 0.7511737089201878, 'Specificity': 0.9862775415539234, 'Recall': 0.7511737089201878, 'Precision': 0.8184143222506394, 'Accuracy': 0.9683928571428572, 'F-score': 0.783353